In [ ]:
#Include
using Plots, LightGraphs, SparseArrays, SimpleWeightedGraphs
using Statistics, BenchmarkTools, LinearAlgebra, ProgressMeter
using Distributions, Base.Threads
using Base.GC
plotly();

# Helpers

In [ ]:
function ConstructWangCC(A,delta::Float64=0.05,epsi::Float64=0.01)
    n = size(A,1)

    defaultval = log((1-delta)/(1+delta))
    C = defaultval*ones(n,n)
    d = sum(A,dims=2)

    # The adjacency matrix squared gives us nodes within a two hop neighborhood
    # This is useful for computing Jaccard coefficients
    A2 = A^2

    for i = 1:n
        # Consider all nodes two hops away
        Ni = findnz(A2[i,:])[1]

        for nj = 1:length(Ni)
            j = Ni[nj]
            if i != j
                # jaccard coefficient:
                numerator = A2[i,j]
                denominator = d[i] + d[j] - numerator

                score = numerator/denominator
                C[i,j] = log((1+score-delta)/(1-score+delta))
                if C[i,j] > 0.0
                    C[i,j] += epsi
                elseif C[i,j] < 0.0
                    C[i,j] -= epsi
                elseif C[i,j] == 0.0 && A[i,j] == 1
                    C[i,j] = epsi
                else
                    C[i,j] = -epsi
                end
            end
        end
    end

    Dmat = zeros(Float64,n,n)

    I = Vector{Int64}()
    J = Vector{Int64}()
    for i = 1:n-1
        for j = i+1:n
            if C[i,j] > 0
               # Anew[j,i] = 1
               # Anew[j,i] = 1
               push!(I,i)
               push!(J,j)
               Dmat[j,i] = 0
               Dmat[i,j] = 0
           else
               Dmat[j,i] = 1
               Dmat[i,j] = 1
            end
        end
    end
    Amat = sparse(I,J,vec(ones(length(I),1)),n,n)

    # Return the adjacency matrix for the positive edges, the weights matrix
    # and the anti-adjacency matrix corresponding to "dissimilarity" scores
    # for the metric nearness formulation
    return Amat, C, Dmat
end

struct MultipleDijkstraState{T <: Real,U <: Integer}
    dists::Array{T,2}
    parents::Array{U,2}
end

function parallel_dp_shortest_paths(g::AbstractGraph{U},
    distmx::AbstractMatrix{T}) where T <: Real where U

    n_v = nv(g)

    # TODO: remove `Int` once julialang/#23029 / #23032 are resolved
    dists   = Array{T,2}(undef,(Int(n_v), Int(n_v)))
    parents = Array{U,2}(undef,(Int(n_v), Int(n_v)))

    Threads.@threads for i in 1:n_v
        state = LightGraphs.dijkstra_shortest_paths(g,[i],distmx)
        dists[i, :] = state.dists
        parents[i, :] = state.parents
    end

    result = MultipleDijkstraState(dists, parents)
    return result
end

function enumerate_paths3(s, v::Integer, i::Integer) where T where U<:Integer
    #pathinfo = s[v,:]
    path = Vector{Integer}()
    currpathindex = i
    while currpathindex != 0
        push!(path, currpathindex)
        if s[v,currpathindex] == currpathindex
            currpathindex = 0
        else
            currpathindex = s[v,currpathindex]
        end
    end
    return path
end

function enumerate_paths2p(s)
    P = Array{Any,1}(undef,size(s.parents, 1))
    Threads.@threads for v = 1:size(s.parents, 1)
        P[v] = LightGraphs.enumerate_paths(s, v)
    end
    
    return P
end

# Ruggles et al code

In [ ]:
include("MetricOpt_helper.jl")
include("ParallelCC.jl")
include("ParallelMetricOpt_1.0.jl")

# Our Algorithm ($K_n$)

In [ ]:
function BregmanCC(D,W⁻¹,C,γ,tol)
    (n,n) = size(D)
    g = LightGraphs.SimpleGraphs.CompleteGraph(n)
    G = copy(D)
    Z = Dict()
    Z′ = zeros(n,n)
    Λ = zeros(n,n)
    Π = zeros(n,n)
    F = -1*γ*(W⁻¹.*C)
    
    FS = LightGraphs.Parallel.floyd_warshall_shortest_paths(g,G) 
    U = FS.dists
    P = enumerate_paths2p(FS)

    maxD2 = norm(U - weights(g),1)
    maxD = tol+1
    max3D = 0
        
    #Project onto new broken cycles
    for i = 1:n
        for j = 1:i-1
            if G[j,i] - U[j,i] > 0  
                p = P[j][i] #enumerate_paths2(FS,i,j)
                l = length(p)
                u = p[1]
                v = p[l]
                d = -1*G[u,v]
                m = W⁻¹[u,v]
                for k = 1:l-1
                    d = d + G[p[k], p[k+1]]
                    m += W⁻¹[p[k], p[k+1]]
                end
                if d < 0
                    c=d/m
                    for k = 1:l-1
                        G[p[k],p[k+1]] -= W⁻¹[p[k],p[k+1]]*c
                        G[p[k+1],p[k]] -= W⁻¹[p[k],p[k+1]]*c
                    end
                    G[p[1],p[l]] += W⁻¹[p[1],p[l]]*c
                    G[p[l],p[1]] += W⁻¹[p[1],p[l]]*c
                    if haskey(Z,p)
                        Z[p] = Z[p] - c
                    else
                        Z[p] = -1*c
                    end
                    if abs(d) > maxD
                        maxD = abs(d)
                    end
                end
            end   
        end
    end
    
    @show((maxD,maxD2,length(Z)))
    flush(stdout)
    
    count = 1
    while(maxD > tol)
        @time maxD = loop(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,C,γ,count)
        flush(stdout)
        count+=1
    end
    return (g,count,Z,F,G - D)
end

function loop(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,C,γ,count)
    for k = 1:2
        #Project onto the cycles we have
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*G[u,v]
            m = W⁻¹[u,v]
            for i = 1:l-1
                d = d + G[p[i], p[i+1]]
                m += W⁻¹[p[i], p[i+1]]
            end
            c = min(d/m,z)
            for i = 1:l-1
                ch = W⁻¹[p[i],p[i+1]]*c
                G[p[i],p[i+1]] -= ch
                G[p[i+1],p[i]] -= ch
            end
            ch = W⁻¹[u,v]*c
            G[u,v] += ch
            G[v,u] += ch
            if z == c
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    end
    
    h1(G,Z′)

    #Find new broken cycles
    FS = LightGraphs.Parallel.floyd_warshall_shortest_paths(g,G) 
    U = FS.dists
    P = enumerate_paths2p(FS)
    
    maxD2 = norm(U - G,1)
    maxD = maximum(G-U)
        
    if count%20 == 0
        max3D = FullTriangleCheck(Base.round.(G,digits=5))
        @show((max3D))
        flush(stdout)
    end
    
    #Project onto new broken cycles
    for i = 1:n
        for j = 1:i-1
            if G[j,i] - U[j,i] > 0  
                p = P[j][i] #enumerate_paths2(FS,i,j)
                l = length(p)
                u = p[1]
                v = p[l]
                d = -1*G[u,v]
                m = W⁻¹[u,v]
                for k = 1:l-1
                    d = d + G[p[k], p[k+1]]
                    m += W⁻¹[p[k], p[k+1]]
                end
                if d < 0
                    c=d/m
                    for k = 1:l-1
                        G[p[k],p[k+1]] -= W⁻¹[p[k],p[k+1]]*c
                        G[p[k+1],p[k]] -= W⁻¹[p[k],p[k+1]]*c
                    end
                    G[p[1],p[l]] += W⁻¹[p[1],p[l]]*c
                    G[p[l],p[1]] += W⁻¹[p[1],p[l]]*c
                    if haskey(Z,p)
                        Z[p] = Z[p] - c
                    else
                        Z[p] = -1*c
                    end
                end
            end   
        end
    end
        
    @show((maxD,maxD2,length(Z),count))
    flush(stdout)
        
    #Make sure that F = |E|
    E = G - D
    
    #You would think the γ*W⁻¹ factor cancels out because Fᵢⱼ, Eᵢⱼ have the same W component. Thus when we calculate
    #δᵢⱼ we need to divide by 2*γ*W⁻¹ᵢⱼ. But then we when we add cᵢⱼ we need to multiply by γ*W⁻¹ᵢⱼ but this is only 
    #if c = δ. if c != δ then this factor is still present. But no! It is present from the previous time c = δ
    #Similar reasoning when we do the g >= 0 projections before and for whenever γ appears
    
    h2(F,E,Λ,Π)
    G = E + D  
    
    return maxD
end

function h1(G,Z′)
    (n,n) = size(G)
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            c = min(G[j,i],Z′[j,i])
            G[j,i] -= c
            G[i,j] -= c
            Z′[j,i] -= c            
        end
    end
    return G
end

function h2(F,E,Λ,Π)
    @inbounds Threads.@threads for i = 1:n
        for j = 1:i-1
            δ1 = (F[j,i]+E[j,i])/2
            δ2 = (F[j,i]-E[j,i])/2
            c1 = min(δ1,Λ[j,i])
            c2 = min(δ2,Π[j,i])
            E[j,i] = E[j,i] - c1 + c2
            E[i,j] = E[i,j] - c1 + c2
            F[j,i] = F[j,i] - c1 - c2
            F[i,j] = F[i,j] - c1 - c2
            Λ[j,i] -= c1
            Π[j,i] -= c2
        end
    end
end

# Our Algorithm sparse

In [ ]:
function h1′(G,Z′,Edges)
    @inbounds for e in Edges
        i = min(e.src,e.dst)
        j = max(e.dst,e.src)
        c = min(G[j,i],Z′[j,i])
        G[j,i] -= c
        G[i,j] -= c
        Z′[j,i] -= c            
    end
end

function h2′(F,E,Λ,Π,Edges)
    @inbounds for e in Edges
        i = min(e.src,e.dst)
        j = max(e.dst,e.src)
        δ1 = (F[j,i]+E[j,i])/2
        δ2 = (F[j,i]-E[j,i])/2
        c1 = min(δ1,Λ[j,i])
        c2 = min(δ2,Π[j,i])
        E[j,i] = E[j,i] - c1 + c2
        E[i,j] = E[i,j] - c1 + c2
        F[j,i] = F[j,i] - c1 - c2
        F[i,j] = F[i,j] - c1 - c2
        Λ[j,i] -= c1
        Π[j,i] -= c2
    end
end

function BregmanCC2(D,W⁻¹,C,γ,tol,g,A)
    (n,n) = size(D)
    G = sparse(copy(D))
    Z = Dict()
    Z′ = spzeros(n,n)
    Λ = spzeros(n,n)
    Π = spzeros(n,n)
    F = sparse(-1*γ*(W⁻¹.*C))
    @show(Sys.free_memory()/2^30)
    flush(stdout)
    
    maxD = tol+1
    Edges = collect(edges(g))
    
    count = 1
    while(maxD > tol || count < 3)
        @time maxD = loop2′(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,γ,count,C,A,Edges)
        flush(stdout)
        count+=1
    end
    return (g,count,Z,F,G - D)
end

function loop2′(g,G,Z,Z′,Λ,Π,F,D,W⁻¹,γ,count,C,A,Edges)
    maxD = 0
    @time for k = 1:5*min(count,15)
        #Project onto the cycles we have
        for p in keys(Z)
            z = Z[p]
            l = length(p)
            u = p[1]
            v = p[l]
            d = -1*G[u,v]
            m = W⁻¹[u,v]
            for i = 1:l-1
                d = d + G[p[i], p[i+1]]
                m += W⁻¹[p[i], p[i+1]]
            end
            c = min(d/m,z)
            if abs(c) > maxD
                maxD = abs(c)
            end
            for i = 1:l-1
                ch = W⁻¹[p[i],p[i+1]]*c
                G[p[i],p[i+1]] -= ch
                G[p[i+1],p[i]] -= ch
            end
            ch = W⁻¹[u,v]*c
            G[u,v] += ch
            G[v,u] += ch
            if z == c || z-c <= 1e-14
                delete!(Z,p)
            else
                Z[p] -= c
            end
        end
    end
    
    flush(stdout)
    
    @time c = min.(G,Z′)
    @time G -= c
    @time Z′-= c
    
    @time JS = parallel_dp_shortest_paths(g,G)
    @show(Sys.free_memory()/2^30)
    flush(stdout)
    
    GC.gc()
    P = JS.parents
        
    #p_lock = SpinLock()
    #Add new broken cycles to the list
    @time for e in Edges
        i = min(e.dst,e.src)
        j = max(e.dst,e.src)
        if JS.dists[i,j] < G[i,j]
            p = enumerate_paths3(P,i,j)
            #lock(p_lock)
            if !haskey(Z,p)
                Z[p] = 0
            end 
            #unlock(p_lock)
        end
    end
    
    JS = 0
    U=0
    P=0
    x = 0
    y = 0
    v = 0
    GC.gc()
        
    #Make sure that F = |E|
    E = G - D
    
    #@time c = min.((E+F)/2,Λ)
    #@time E -= c
    #@time F -= c
    #@time Λ -= c
    
    #@time c = min.((F-E)/2,Π)
    #@time E += c
    #@time F -= c
    #@time Π -= c
    
    
    
    #You would think the γ*W⁻¹ factor cancels out because Fᵢⱼ, Eᵢⱼ have the same W component. Thus when we calculate
    #δᵢⱼ we need to divide by 2*γ*W⁻¹ᵢⱼ. But then we when we add cᵢⱼ we need to multiply by γ*W⁻¹ᵢⱼ but this is only 
    #if c = δ. if c != δ then this factor is still present. But no! It is present from the previous time c = δ
    #Similar reasoning when we do the g >= 0 projections before and for whenever γ appears
    @time for r = 1:1
        h2′(F,E,Λ,Π,Edges)
    end
    G = E + D  
    
    obj = sum(W.*F)
    obj2 = sum(W.*(abs.(E)))
    @show((maxD,length(Z), obj, obj2))
    
    return maxD
end

# Run Ruggles et al code

In [ ]:
G = loadgraph("Graphs/power.lgz")
n = nv(G)

A = LightGraphs.LinAlg.adjacency_matrix(G);

# Define a few parameters
GapTol = .01
ConTol = .01
filename = "power-3cycle"
statusFrequency = 10
gam = 1.0
maxits = 1000
lam = .2

# Solve a relaxed Lambda-Correlation Clustering Problem
@time Xlamcc = Parallel_Dykstra_lamCC_TFA(A,GapTol,ConTol,lam,filename,gam,maxits,statusFrequency)

# Run our code ($K_n$)

In [ ]:
G = loadgraph("Graphs/power.lgz")
n = nv(G)

C = LightGraphs.LinAlg.adjacency_matrix(G)
for i = 1:n
    C[i,i] = 0
end
A,C,Dmat = ConstructWangCC(C);

W⁻¹ = 1 ./(abs.(C));

@time R = BregmanCC(Dmat,W⁻¹,C,1,1e-2)

# Run our code (Sparse)

In [ ]:
Gp = loadgraph("Signed Networks/slashdotp.lgz")
Gn = loadgraph("Signed Networks/slashdotn.lgz")

n = nv(Gp)

Cp = LightGraphs.LinAlg.adjacency_matrix(Gp)
Cn = LightGraphs.LinAlg.adjacency_matrix(Gn);

C = Cp - Cn;

Dmat = spzeros(n,n)
@showprogress for i = 1:n
    for j = 1:n
        if C[i,j] == -1.0
            Dmat[i,j] = 1
        end
    end
end

W = abs.(C)
for i = 1:n
    if C[i,i] != 0.0
        W[i,i] = 0
        C[i,i] = 0
        Dmat[i,i] = 0
    end
end 
g = SimpleGraph(W)

@time R = BregmanCC2(Dmat,W,W,1.0,1e-2,g,W)